# Human Value Detection

## Imports

In [ ]:
import os

import transformers
%load_ext autoreload
%autoreload 2

from utilities import *
from models.bertOne import BertOne
from models.berTwo import BerTwo

from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix
from models.randomUniformClassifier import RandomUniformClassifier
from models.majorityCalssifier import MajorityClassifier

from transformers import AutoTokenizer

from drTorch.callbacks import EarlyStopper

from drTorch.metrics import F1_Score
from drTorch.metrics import F1_Score_Multi_Labels
from drTorch.utilities import *
from drTorch.wrappers import OptimizerWrapper
from drTorch.wrappers import Criterion

import numpy as np
import torch
import pandas as pd


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print('Device: %s' % device)


## Defining constants and flags

In [ ]:
# PATHS AND DATAFRAME CREATION
DATA_DIR = "data"
ARGUMENTS_DIR = os.path.join(DATA_DIR, "arguments")
LABELS_DIR = os.path.join(DATA_DIR, "labels")

BERT_MODELS_DIRECTORY = "bert_models"
BERT_VERSIONS=["bert-base-uncased"]

# CONSTANTS 
N_LABELS = 4
N_CLASSES = 2
BATCH_SIZE = 1
#os.environ["TOKENIZERS_PARALLELISM"] = "true"

CLASS_2_ONE_HOT = {class_label: np.eye(N_CLASSES)[i].astype(float).tolist() for i, class_label in enumerate(range(N_CLASSES))}


## Task 1

### Visualizing the data

In [ ]:
# convert files in dataframes
train_arg_df, val_arg_df, test_arg_df = create_dfs(ARGUMENTS_DIR)
train_labels_df, val_labels_df, test_labels_df = create_dfs(LABELS_DIR)

print("Let's visualize the data: ")
display(train_arg_df.head(5))
display(train_labels_df.head(5))


### Mapping labels to level-3 categories

In [ ]:
mapping = define_mapping()
train_labels_df, val_labels_df, test_labels_df = map_to_level_3(mapping, train_labels_df, val_labels_df, test_labels_df) 

print("The training labels after the mapping are the following: ")
train_labels_df


### One-hot encoding, tokenization and data loaders building

In [ ]:
if os.path.exists(BERT_MODELS_DIRECTORY):
    bert_versions_paths = os.listdir(BERT_MODELS_DIRECTORY)
else:    
    bert_versions_paths = download_bert_models(BERT_MODELS_DIRECTORY, BERT_VERSIONS)  
    

In [1]:
model_inputs = {"C":["Conclusion"], "CP":["Conclusion", "Premise"], "CPS":["Conclusion", "Premise", "Stance"]}

dataloader_train_C, dataloader_val_C, dataloader_test_C  = build_dataloaders(train_df=train_arg_df, 
                                                                             val_df=val_arg_df, 
                                                                             test_df=test_arg_df, 
                                                                             train_labels_df=train_labels_df, 
                                                                             val_labels_df=val_labels_df, 
                                                                             test_labels_df=test_labels_df, 
                                                                             one_hot_mapping=CLASS_2_ONE_HOT, 
                                                                             bert_version="bert-base-uncased", 
                                                                             model_input=model_inputs["C"],
                                                                             custom_dataset_builder=CustomDataset_C,
                                                                             batch_size=BATCH_SIZE, 
                                                                             shuffle=True)

dataloader_train_CP, dataloader_val_CP, dataloader_test_CP  = build_dataloaders(train_df=train_arg_df, 
                                                                                val_df=val_arg_df, 
                                                                                test_df=test_arg_df, 
                                                                                train_labels_df=train_labels_df, 
                                                                                val_labels_df=val_labels_df, 
                                                                                test_labels_df=test_labels_df, 
                                                                                one_hot_mapping=CLASS_2_ONE_HOT, 
                                                                                bert_version="bert-base-uncased", 
                                                                                model_input=model_inputs["CP"],
                                                                                custom_dataset_builder=CustomDataset_CP,
                                                                                batch_size=BATCH_SIZE, 
                                                                                shuffle=True)

dataloader_train_CPS, dataloader_val_CPS, dataloader_test_CPS  = build_dataloaders(train_df=train_arg_df, 
                                                                                   val_df=val_arg_df, 
                                                                                   test_df=test_arg_df, 
                                                                                   train_labels_df=train_labels_df, 
                                                                                   val_labels_df=val_labels_df, 
                                                                                   test_labels_df=test_labels_df, 
                                                                                   one_hot_mapping=CLASS_2_ONE_HOT, 
                                                                                   bert_version="bert-base-uncased", 
                                                                                   model_input=model_inputs["CPS"],
                                                                                   custom_dataset_builder=CustomDataset_CPS,
                                                                                   batch_size=BATCH_SIZE, 
                                                                                   shuffle=True)


NameError: name 'build_dataloaders' is not defined

## Task 2 

### Models Definition

####  1) Random uniform classifier

In [ ]:
# Create an instance of the random uniform classifier
random_classifier = RandomUniformClassifier(N_LABELS)

# Make predictions on the test set
predicted_labels = random_classifier.predict(test_arg_df)

# Accuracy of the Random Classifier
accuracy = accuracy_score(test_labels_df, predicted_labels)
print(f'Accuracy of the model over all the classes: {accuracy}\n')

# F1 scores on the different labels singularly taken 
f1_scorer = F1_Score_Multi_Labels(name='F1_macro_avg', num_labels=N_LABELS, num_classes=N_CLASSES, compute_mean=False)
f1_scores_random_classifier = f1_scorer(torch.tensor(test_labels_df.values.tolist()), torch.tensor(predicted_labels))
# AVG f1 score
f1_avg_random_classifier = np.average(f1_scores_random_classifier)


for idx, f1_score in enumerate(f1_scores_random_classifier):
    print(f" - {train_labels_df.columns[idx]} F1: {f1_score}")

print(f'\nAverage F1: {f1_avg_random_classifier}')

####  1) Majority classifier

In [ ]:
majority_classifier = MajorityClassifier()

# Train the majority classifier (even though in practice, no training is needed)
majority_classifier.fit(train_labels_df)

# Make predictions on the test set
predicted_labels = majority_classifier.predict(test_labels_df)

# Accuracy of the Majority Classifier
accuracy = accuracy_score(test_labels_df, predicted_labels)
print(f'Accuracy of the model over all the classes: {accuracy}\n')

# F1 scores on the different labels singularly taken 
f1_scorer = F1_Score_Multi_Labels(name='F1_macro_avg', num_labels=N_LABELS, num_classes=N_CLASSES, compute_mean=False)
f1_scores_majority_classifier = f1_scorer(torch.tensor(test_labels_df.values.tolist()), torch.tensor(predicted_labels))
# AVG F1 score
f1_avg_majority_classifier = np.average(f1_scores_majority_classifier)


for idx, f1_score in enumerate(f1_scores_majority_classifier):
    print(f" - {train_labels_df.columns[idx]} F1: {f1_score}")

print(f'\nAverage F1: {f1_avg_majority_classifier}')


####  3) Bert w/C

In [ ]:
optimizer_test = OptimizerWrapper(torch.optim.Adam, identifier=f'lr={1e-5}', optimizer_partial_params={'lr': 1e-6})          
criterion_test = Criterion('loss', loss_function=torch.nn.BCEWithLogitsLoss(reduction='none'), reduction_function=torch.sum)

bert1 = BertOne(dropout_prob=0.3, hidden_size= 768, bert_version='bert-base-uncased').to(device)

bert1_history = bert1.fit(train_loader=dataloader_train_C, 
                          val_loader=dataloader_val_C, 
                          criterion=criterion_test, 
                          metrics=[F1_Score_Multi_Labels('F1_macro_avg', num_labels=N_LABELS, num_classes=N_CLASSES)], 
                          optimizer=optimizer_test,
                          # early_stopper=EarlyStopper(monitor='F1_macro', patience=4, delta=0, mode='max', restore_weights=True),
                          num_epochs=200)

"""
loss homework 1 con batch size 1
[0.4, 0.5, 0.8, ... 0.2] 46 loss

loss homework 2 con batch size 1
        [[0.4339, 0.4490],
        [0.5538, 0.6676],
        [0.4004, 0.3562],
        [0.6964, 0.6620]]
"""

#### 4) Bert w/CP

In [ ]:
optimizer_test = OptimizerWrapper(torch.optim.Adam, identifier=f'lr={1e-6}', optimizer_partial_params={'lr': 1e-6})          
criterion_test = Criterion('loss', loss_function=torch.nn.BCEWithLogitsLoss(reduction='none'), reduction_function=torch.mean)

bert2 = BerTwo(dropout_prob=0.3, hidden_size= 768, bert_version='bert-base-uncased').to(device)

bert2_history = bert2.fit(train_loader=dataloader_train_CP, 
                          val_loader=dataloader_val_CP, 
                          criterion=criterion_test, 
                          metrics=[F1_Score_Multi_Labels('F1_macro_avg', num_labels=N_LABELS, num_classes=N_CLASSES)], 
                          optimizer=optimizer_test,
                          # early_stopper=EarlyStopper(monitor='F1_macro', patience=4, delta=0, mode='max', restore_weights=True),
                          num_epochs=200)

In [ ]:
### Test sul modello

tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
config=transformers.BertConfig.from_pretrained('bert-base-uncased')
text = "Replace me by any text you'd like pollo."
encoded_input = tokenizer(text, return_tensors='pt')

model = transformers.BertModel.from_pretrained("bert-base-uncased")
output = model(**encoded_input)
print(model.config)
"""p_d = BertOne()
output1 = p_d(**encoded_input)"""


In [ ]:
from torch.nn.functional import max_pool2d

print(output[0].shape)
output_2 = max_pool2d(output[0], kernel_size=(14,1))

output_2[:,0,:].shape